<a href="https://colab.research.google.com/github/RobertCall/FakeNewsNet/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pymorphy2
!pip install ufal.udpipe
!pip install corpy
!pip install -U pymorphy2-dicts-ru

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 8.2 MB 10.8 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=ed3bcbd120696b8d5c51ed621b56715227f591d5b3c32fc43cc533d0a372475c
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 304 kB 5.0 MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp38-cp38-linux_x86_64.whl size=5626903 sha256=887ecc587c8004705a9c864eb9c10fa0ef3781ccf4b87608beb6075140782546
  Stored in directory: /root/.cache/pip/wheels/d4/c1/67/142cea91540458ab9edac9c280a19b549a03217d7b441d32a6
Successfully built ufal.udpipe
Looking in indexes: https://pypi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pymorphy2

import os
import gensim

import pandas as pd

import ufal.udpipe as udp
import corpy.udpipe as crp

from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten



In [10]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
nltk_tokenizer = RegexpTokenizer(r'[а-яёa-z]+')

morph = pymorphy2.MorphAnalyzer()

def text_preprocessing(text):
  words = nltk_tokenizer.tokenize(text.lower())
  lem_text = [morph.parse(w)[0].normal_form for w in words if w not in stop_words]

  return lem_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
if not os.path.isfile('220.zip'):
  !wget http://vectors.nlpl.eu/repository/20/220.zip
  !unzip 220.zip

w2v = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

--2022-12-26 10:34:55--  http://vectors.nlpl.eu/repository/20/220.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638171816 (609M) [application/zip]
Saving to: ‘220.zip’

220.zip             100%[===================>] 608.61M  18.6MB/s    in 35s     

2022-12-26 10:35:30 (17.5 MB/s) - ‘220.zip’ saved [638171816/638171816]

Archive:  220.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [8]:
#df_news = pd.read_csv ('news0.csv')
df_news = pd.read_csv ('train.tsv', sep='\t')
test = pd.read_csv ('test.tsv', sep='\t')

In [ ]:
# Скачивание модели UDPipe, обученную на русском языке
udp_model_url = r'https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-syntagrus-ud-2.5-191206.udpipe'
udp_model_filename = 'russian-syntagrus-ud-2.5-191206.udpipe'
#if not os.path.isfile(udp_model_filename):
 # !wget.download(udp_model_url)

# Загрузка модели в оболочку corpy
# corpy_model = udp.Model.load(udp_model_filename)
corpy_model = crp.Model(udp_model_filename)
print('model', corpy_model)

# Функция для тегирования слов
def udp_tagging(lem_text):
  sents = [list(corpy_model.process(w)) for w in lem_text]
  tagged_words = [s[0].words[1].form + '_' + s[0].words[1].upostag for s in sents if s]

  return tagged_words

model <corpy.udpipe.Model object at 0x7f54d2de9e80>


In [ ]:
w2v.most_similar(udp_tagging(['учитель'])) #Тестируем w2v на слове "учитель"

[('учительница_NOUN', 0.7843227386474609),
 ('ученик_NOUN', 0.7229872941970825),
 ('преподаватель_NOUN', 0.6898813843727112),
 ('воспитатель_NOUN', 0.667870283126831),
 ('учитель_PROPN', 0.609573245048523),
 ('репетитор_NOUN', 0.6008316278457642),
 ('учить_VERB', 0.5971025228500366),
 ('завуч_NOUN', 0.5949769020080566),
 ('урок_NOUN', 0.5922257304191589),
 ('преподавательница_NOUN', 0.5865318775177002)]

In [ ]:
import pickle

X_not_filtered_texts = df_news['title'].apply(lambda x: udp_tagging(text_preprocessing(x))).array
with open('data1.pickle', 'wb') as f:
    pickle.dump(X_not_filtered_texts, f)
#with open ('data.pickle', 'rb') as f:
   #X_not_filtered_texts = pickle.load(f)

X_filtered_texts = []
for text in X_not_filtered_texts:
  words = []
  for word in text:
    if word in w2v.vocab:
      words.append(word)
  if len(words) == 0:
    print ("asd")
  X_filtered_texts.append(words)

In [ ]:
X = [list(map(lambda x: float(w2v.vocab[x].index), t)) for t in X_filtered_texts]
X = pad_sequences(X, maxlen=100)
X = np.asarray(X).astype('float32')

Y = np.array(df_news['is_fake']).astype('float32').reshape((-1,1))

x_train, x_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
seq_model = Sequential()

weights = w2v.vectors
layer = Embedding(
    input_dim=weights.shape[0],
    output_dim=weights.shape[1],
    weights=[weights],
    input_length=100,
    mask_zero=True,
    trainable=False,
)

seq_model.add(layer)
seq_model.add(Dense(50, activation='relu'))
seq_model.add(Flatten())
seq_model.add(Dropout(0.6))
seq_model.add(Dense(1, activation='sigmoid'))

seq_model.compile(loss='mean_squared_logarithmic_error',
                  optimizer='adam', metrics=['accuracy'])

seq_model.fit(x_train, y_train, epochs=20,
               validation_data=(x_test, y_test))

seq_model.summary()
# seq_model.save("model.keras")

Epoch 1/20
135/135 [==============================] - 3s 19ms/step - loss: 0.1616 - accuracy: 0.6042 - val_loss: 0.0755 - val_accuracy: 0.7715
Epoch 2/20
135/135 [==============================] - 2s 15ms/step - loss: 0.0778 - accuracy: 0.7649 - val_loss: 0.0648 - val_accuracy: 0.8111
Epoch 3/20
135/135 [==============================] - 2s 15ms/step - loss: 0.0680 - accuracy: 0.7990 - val_loss: 0.0618 - val_accuracy: 0.8174
Epoch 4/20
135/135 [==============================] - 2s 16ms/step - loss: 0.0626 - accuracy: 0.8194 - val_loss: 0.0601 - val_accuracy: 0.8146
Epoch 5/20
135/135 [==============================] - 2s 15ms/step - loss: 0.0595 - accuracy: 0.8247 - val_loss: 0.0573 - val_accuracy: 0.8299
Epoch 6/20
135/135 [==============================] - 2s 15ms/step - loss: 0.0562 - accuracy: 0.8351 - val_loss: 0.0574 - val_accuracy: 0.8306
Epoch 7/20
135/135 [==============================] - 2s 18ms/step - loss: 0.0536 - accuracy: 0.8430 - val_loss: 0.0554 - val_accuracy: 0.8347

In [ ]:
from tensorflow.keras.models import load_model
seq_model = load_model("model.keras")

In [6]:
fake_texts = ['Новак спрогнозировал новый мировой энергокризис через 5-10 лет',
              'Во всех школах России установят счетчик американского госдолга',
              'РФ продолжает рассматривать европу как потенциальный рынок для сбыта газа',
              'Число погибших от последствий зимнего шторма в США выросло до 28',
              'В магазинах появились дешевые яйца без желтков']

In [ ]:
fake_preproc = [udp_tagging(text_preprocessing(fake_text)) for fake_text in fake_texts]

fake_words_tagged = []
for text in fake_preproc:
  words = []
  for word in text:
    if word in w2v.vocab:
      words.append(word)
  fake_words_tagged.append(words)

fake_indexed = [list(map(lambda x: float(w2v.vocab[x].index), t)) for t in fake_words_tagged]
fake_indexed = pad_sequences(fake_indexed, maxlen=100)
fake_indexed = np.asarray(fake_indexed).astype('float32')

print(seq_model.predict(fake_indexed))

1/1 [==============================] - 0s 23ms/step
[[0.37466243]
 [0.6659432 ]
 [0.11183485]
 [0.7457472 ]
 [0.4915185 ]]


In [ ]:
test = pd.read_csv ('test.tsv', sep='\t')
fake_preproc = test['title'].apply(lambda x: udp_tagging(text_preprocessing(x))).array
fake_words_tagged = []
for text in fake_preproc:
  words = []
  for word in text:
    if word in w2v.vocab:
      words.append(word)
  fake_words_tagged.append(words)
Y = np.array(test['is_fake']).astype('float32').reshape((-1,1))

fake_indexed = [list(map(lambda x: float(w2v.vocab[x].index), t)) for t in fake_words_tagged]
fake_indexed = pad_sequences(fake_indexed, maxlen=100)
fake_indexed = np.asarray(fake_indexed).astype('float32')

print(seq_model.evaluate(fake_indexed, Y))


32/32 [==============================] - 0s 7ms/step - loss: 0.2039 - accuracy: 0.5410
[0.2038509100675583, 0.5410000085830688]


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import PassiveAggressiveClassifier
fake_preproc = df_news['title'].apply(lambda x: " ".join(text_preprocessing(x))).array
Y = np.array(df_news['is_fake']).astype('float32').reshape((-1,1))
x_train, x_test, y_train, y_test = train_test_split(fake_preproc, Y)
tfidf = TfidfVectorizer()
vec_train = tfidf.fit_transform(x_train)
vec_val = tfidf.transform(x_test)

pac = PassiveAggressiveClassifier(C = 0.01)

pac.fit(vec_train, y_train)
val_pred = pac.predict(vec_val)


print(classification_report(y_test, val_pred))
fake_preproc = test['title'].apply(lambda x: " ".join(text_preprocessing(x))).array
Y = np.array(test['is_fake']).astype('float32').reshape((-1,1))
vec_val = tfidf.transform(fake_preproc)
val_pred = pac.predict(vec_val)
print(classification_report(Y, val_pred))


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

         0.0       0.85      0.81      0.83       735
         1.0       0.81      0.85      0.83       705

    accuracy                           0.83      1440
   macro avg       0.83      0.83      0.83      1440
weighted avg       0.83      0.83      0.83      1440

              precision    recall  f1-score   support

         0.0       1.00      0.49      0.65      1000
         1.0       0.00      0.00      0.00         0

    accuracy                           0.49      1000
   macro avg       0.50      0.24      0.33      1000
weighted avg       1.00      0.49      0.65      1000



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
fake_preproc = [" ".join(text_preprocessing(fake_text)) for fake_text in fake_texts]
vec_val = tfidf.transform(fake_preproc)
val_pred = pac.decision_function(vec_val)
val_pred0 = pac.predict(vec_val)
print(val_pred, val_pred0)

[-0.79722638  1.0119444  -1.03961405 -0.4323088   0.33548233] [0. 1. 0. 0. 1.]
